## Исследование базы данных вопросов и ответов StackOverflow

**Цель проекта:**

Решать поставленые задачи, с помощью SQL запросов. 

**Описание проекта:**

Задачи автоматически проверяются в тренажере. Самостоятельная работа основана на базе данных StackOverflow — сервиса вопросов и ответов о программировании с данными о постах за 2008 год.

**Инструментарий:**
* PostgreSQL
* Oконные функции, подзапросы, JOIN-соединения

## Задача 1. 
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [1]:
'''
SELECT 
      COUNT(*)
FROM 
      stackoverflow.posts
WHERE 
     post_type_id = 1 AND (score > 300 OR favorites_count >=100)
'''

'\nSELECT \n      COUNT(*)\n\nFROM \n      stackoverflow.posts\nWHERE \n     post_type_id = 1 AND (score > 300 OR favorites_count >=100)\n'

## Задача 2. 
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [3]:
'''
SELECT
      ROUND(AVG(t.avs))

FROM

(SELECT
      CAST(DATE_TRUNC('DAY', creation_date) AS date),
      COUNT(id) AS avs
      
FROM stackoverflow.posts
WHERE (post_type_id = 1) AND 
      (creation_date :: date BETWEEN '2008.11.01' AND '2008.11.18')
GROUP BY 
      CAST(DATE_TRUNC('DAY', creation_date) AS date)) AS t
'''

"\nSELECT\n      ROUND(AVG(t.avs))\n\nFROM\n\n(SELECT\n      CAST(DATE_TRUNC('DAY', creation_date) AS date),\n      COUNT(id) AS avs\n      \nFROM stackoverflow.posts\nWHERE (post_type_id = 1) AND \n      (creation_date :: date BETWEEN '2008.11.01' AND '2008.11.18')\nGROUP BY \n      CAST(DATE_TRUNC('DAY', creation_date) AS date)) AS t\n"

## Задача 3. 
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [1]:
'''
SELECT COUNT(DISTINCT user_id)

FROM stackoverflow.users AS u
                             LEFT JOIN stackoverflow.badges AS b ON u.id=b.user_id
                             
WHERE b.creation_date :: date = u.creation_date :: date
'''

'\nSELECT COUNT(DISTINCT user_id)\n\nFROM stackoverflow.users AS u\n                             LEFT JOIN stackoverflow.badges AS b ON u.id=b.user_id\n                             \nWHERE b.creation_date :: date = u.creation_date :: date\n'

## Задача 4. 
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [2]:
'''
SELECT
      COUNT(DISTINCT p.id)

FROM stackoverflow.posts AS p                                               
                             JOIN stackoverflow.users AS u ON p.user_id=u.id 
                             JOIN stackoverflow.votes AS v ON p.id = v.post_id 
                             
WHERE v.id > 0 
                    AND u.display_name = 'Joel Coehoorn';
'''

"\nSELECT\n      COUNT(DISTINCT p.id)\n\nFROM stackoverflow.posts AS p                                               \n                             JOIN stackoverflow.users AS u ON p.user_id=u.id \n                             JOIN stackoverflow.votes AS v ON p.id = v.post_id \n                             \nWHERE v.id > 0 \n                    AND u.display_name = 'Joel Coehoorn';\n"

## Задача 5. 
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [3]:
'''
SELECT *,
       ROW_NUMBER() OVER(ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id 
'''

'\nSELECT *,\n       ROW_NUMBER() OVER(ORDER BY id DESC)\nFROM stackoverflow.vote_types\nORDER BY id \n'

## Задача 6. 
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [4]:
'''
SELECT
      *
FROM (SELECT v.user_id AS c,
       COUNT(vt.id) AS b

FROM stackoverflow.votes AS v 
                             JOIN stackoverflow.vote_types AS vt ON v.vote_type_id=vt.id                           
WHERE vt.name = 'Close'
GROUP BY v.user_id
ORDER BY COUNT(v.user_id) DESC
LIMIT 10) AS D

ORDER BY 
        D.b DESC,
        D.c DESC;
'''

"\nSELECT\n      *\nFROM (SELECT v.user_id AS c,\n       COUNT(vt.id) AS b\n\nFROM stackoverflow.votes AS v \n                             JOIN stackoverflow.vote_types AS vt ON v.vote_type_id=vt.id                           \nWHERE vt.name = 'Close'\nGROUP BY v.user_id\nORDER BY COUNT(v.user_id) DESC\nLIMIT 10) AS D\n\nORDER BY \n        D.b DESC,\n        D.c DESC;\n"

## Задача 7. 
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [5]:
'''
SELECT
      use,
      bage,
      DENSE_RANK() OVER(ORDER BY bage DESC)
      
FROM 
(SELECT 
      user_id AS use,
      COUNT(id) AS bage
FROM stackoverflow.badges
WHERE creation_date :: date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY bage DESC, user_id LIMIT 10) AS tabl
'''

"\nSELECT\n      use,\n      bage,\n      DENSE_RANK() OVER(ORDER BY bage DESC)\n      \nFROM \n(SELECT \n      user_id AS use,\n      COUNT(id) AS bage\nFROM stackoverflow.badges\nWHERE creation_date :: date BETWEEN '2008-11-15' AND '2008-12-15'\nGROUP BY user_id\nORDER BY bage DESC, user_id LIMIT 10) AS tabl\n"

## Задача 8. 
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [6]:
'''
SELECT *,
      ROUND(AVG(score) OVER(PARTITION BY user_id))

FROM
(SELECT
      title,
      user_id,
      score

FROM stackoverflow.posts AS p
WHERE title IS NOT NULL AND score != 0) A
'''

'\nSELECT *,\n      ROUND(AVG(score) OVER(PARTITION BY user_id))\n\nFROM\n(SELECT\n      title,\n      user_id,\n      score\n\nFROM stackoverflow.posts AS p\nWHERE title IS NOT NULL AND score != 0) A\n'

## Задача 9. 
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [7]:
'''
SELECT
      title
FROM 
      stackoverflow.posts
      
WHERE user_id IN (
  
SELECT 
      user_id
FROM stackoverflow.badges

GROUP BY 1
HAVING COUNT(id) > 1000) AND title IS NOT NULL 
'''

'\nSELECT\n      title\nFROM \n      stackoverflow.posts\n      \nWHERE user_id IN (\n  \nSELECT \n      user_id\nFROM stackoverflow.badges\n\nGROUP BY 1\nHAVING COUNT(id) > 1000) AND title IS NOT NULL \n'

## Задача 10. 
Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [8]:
'''
SELECT id,
      views,
        CASE
    WHEN views>=350 THEN '1'
    WHEN views<350 AND views>=100 THEN '2'
    WHEN views<100 THEN '3'
         END :: numeric
  
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views > 0 
'''

"\nSELECT id,\n      views,\n        CASE\n    WHEN views>=350 THEN '1'\n    WHEN views<350 AND views>=100 THEN '2'\n    WHEN views<100 THEN '3'\n         END :: numeric\n  \nFROM stackoverflow.users\nWHERE location LIKE '%United States%' AND views > 0 \n"

## Задача 11. 
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [9]:
'''
WITH
a AS (SELECT id,
      views,
        CASE
    WHEN views>=350 THEN 1
    WHEN views<350 AND views>=100 THEN 2
    WHEN views<100 THEN 3
         END AS ccc
  
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views > 0
ORDER BY 2 DESC,3),

b AS (
SELECT 
      ccc,
      MAX(views) as mmm
FROM a
GROUP BY 1)

SELECT DISTINCT
     a.id,
     b.ccc,
     b.mmm

FROM b LEFT JOIN a ON b.mmm = a.views

ORDER BY mmm DESC, id
'''

"\nWITH\na AS (SELECT id,\n      views,\n        CASE\n    WHEN views>=350 THEN 1\n    WHEN views<350 AND views>=100 THEN 2\n    WHEN views<100 THEN 3\n         END AS ccc\n  \nFROM stackoverflow.users\nWHERE location LIKE '%United States%' AND views > 0\nORDER BY 2 DESC,3),\n\nb AS (\nSELECT \n      ccc,\n      MAX(views) as mmm\nFROM a\nGROUP BY 1)\n\nSELECT DISTINCT\n     a.id,\n     b.ccc,\n     b.mmm\n\nFROM b LEFT JOIN a ON b.mmm = a.views\n\nORDER BY mmm DESC, id\n"

## Задача 12. 
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [10]:
'''
WITH

A as
(SELECT 
      EXTRACT(DAY FROM CAST(creation_date AS date)) AS day_count,
      COUNT(id) AS user_count
      
FROM stackoverflow.users
 
WHERE creation_date :: date BETWEEN '2008-11-1' AND '2008-11-30'

GROUP BY 1)

SELECT
      day_count,
      user_count,
      SUM(user_count) OVER(ORDER BY day_count)
      
FROM a
'''

"\nWITH\n\nA as\n(SELECT \n      EXTRACT(DAY FROM CAST(creation_date AS date)) AS day_count,\n      COUNT(id) AS user_count\n      \nFROM stackoverflow.users\n \nWHERE creation_date :: date BETWEEN '2008-11-1' AND '2008-11-30'\n\nGROUP BY 1)\n\nSELECT\n      day_count,\n      user_count,\n      SUM(user_count) OVER(ORDER BY day_count)\n      \nFROM a\n"

## Задача 13. 

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [11]:
'''
SELECT DISTINCT
       p.user_id ,
       FIRST_VALUE(p.creation_date) OVER(PARTITION BY p.user_id  ORDER BY p.creation_date) - u.creation_date

FROM stackoverflow.users AS u
                             JOIN stackoverflow.posts AS p ON u.id = p.user_id 
'''

'\nSELECT DISTINCT\n       p.user_id ,\n       FIRST_VALUE(p.creation_date) OVER(PARTITION BY p.user_id  ORDER BY p.creation_date) - u.creation_date\n\nFROM stackoverflow.users AS u\n                             JOIN stackoverflow.posts AS p ON u.id = p.user_id \n'

## Задача 14. 

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [12]:
'''
SELECT 
      CAST(DATE_TRUNC('MONTH', creation_date) AS date),
      SUM(views_count)
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM CAST(creation_date AS date)) = 2008
GROUP BY CAST(DATE_TRUNC('MONTH', creation_date) AS date)
ORDER BY 2 DESC
'''

"\nSELECT \n      CAST(DATE_TRUNC('MONTH', creation_date) AS date),\n      SUM(views_count)\nFROM stackoverflow.posts\nWHERE EXTRACT(YEAR FROM CAST(creation_date AS date)) = 2008\nGROUP BY CAST(DATE_TRUNC('MONTH', creation_date) AS date)\nORDER BY 2 DESC\n"

## Задача 15. 


Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [13]:
'''
SELECT 
      u.display_name,
      COUNT(DISTINCT u.id)

FROM stackoverflow.users AS u 
                               JOIN stackoverflow.posts AS p ON u.id = p.user_id
                                       
WHERE p.creation_date :: date <= u.creation_date :: date + INTERVAL '1 months'
      AND post_type_id = 2

GROUP BY u.display_name
HAVING COUNT(post_type_id) > 100
'''

"\nSELECT \n      u.display_name,\n      COUNT(DISTINCT u.id)\n\nFROM stackoverflow.users AS u \n                               JOIN stackoverflow.posts AS p ON u.id = p.user_id\n                                       \nWHERE p.creation_date :: date <= u.creation_date :: date + INTERVAL '1 months'\n      AND post_type_id = 2\n\nGROUP BY u.display_name\nHAVING COUNT(post_type_id) > 100\n"

## Задача 16. 

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [14]:
'''
WITH users AS (SELECT u.id
               FROM stackoverflow.posts AS p
               JOIN stackoverflow.users AS u ON p.user_id=u.id
               WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01' 
                   AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
               GROUP BY u.id
               HAVING COUNT(p.id) > 0)

SELECT COUNT(p.id),
       DATE_TRUNC('month', p.creation_date)::date
FROM stackoverflow.posts AS p
WHERE p.user_id IN (SELECT *
                    FROM users)
      AND DATE_TRUNC('year', p.creation_date)::date = '2008-01-01'
GROUP BY DATE_TRUNC('month', p.creation_date)::date
ORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;
'''

"\nWITH users AS (SELECT u.id\n               FROM stackoverflow.posts AS p\n               JOIN stackoverflow.users AS u ON p.user_id=u.id\n               WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01' \n                   AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'\n               GROUP BY u.id\n               HAVING COUNT(p.id) > 0)\n\nSELECT COUNT(p.id),\n       DATE_TRUNC('month', p.creation_date)::date\nFROM stackoverflow.posts AS p\nWHERE p.user_id IN (SELECT *\n                    FROM users)\n      AND DATE_TRUNC('year', p.creation_date)::date = '2008-01-01'\nGROUP BY DATE_TRUNC('month', p.creation_date)::date\nORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;\n"

## Задача 17. 

Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.


Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [16]:
'''
SELECT
      user_id,
      creation_date,
      views_count,
      SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)

FROM stackoverflow.posts
'''

'\nSELECT\n      user_id,\n      creation_date,\n      views_count,\n      SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)\n\nFROM stackoverflow.posts\n'

## Задача 18. 

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат

In [18]:
'''
WITH ad AS (SELECT user_id,
                   COUNT(DISTINCT creation_date::date)  AS cnt
            FROM stackoverflow.posts
            WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' 
            GROUP BY user_id)

SELECT ROUND(AVG(ad.cnt))
FROM ad;
'''

"\nWITH ad AS (SELECT user_id,\n                   COUNT(DISTINCT creation_date::date)  AS cnt\n            FROM stackoverflow.posts\n            WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' \n            GROUP BY user_id)\n\nSELECT ROUND(AVG(ad.cnt))\nFROM ad;\n"

## Задача 19. 

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.


Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [19]:
'''
SELECT
      EXTRACT(MONTH FROM CAST(creation_date AS date)),
      COUNT(id) AS zzz,
      ROUND(((COUNT(id) :: numeric / LAG(COUNT(id)) OVER()) - 1) * 100,2)
      
      
FROM stackoverflow.posts

WHERE CAST(DATE_TRUNC('MONTH', creation_date) AS date) BETWEEN '2008-09-01' AND '2008-12-31'

GROUP BY EXTRACT(MONTH FROM CAST(creation_date AS date))
'''

"\nSELECT\n      EXTRACT(MONTH FROM CAST(creation_date AS date)),\n      COUNT(id) AS zzz,\n      ROUND(((COUNT(id) :: numeric / LAG(COUNT(id)) OVER()) - 1) * 100,2)\n      \n      \nFROM stackoverflow.posts\n\nWHERE CAST(DATE_TRUNC('MONTH', creation_date) AS date) BETWEEN '2008-09-01' AND '2008-12-31'\n\nGROUP BY EXTRACT(MONTH FROM CAST(creation_date AS date))\n"

## Задача 20. 

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.


Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.